In [ ]:
# Imports
import requests
import pandas as pd
import json
from datetime import datetime
import urllib.parse

In [ ]:
# Define date to add it to the filename of the generated files
date=datetime.today().strftime('%Y.%m.%d')
print('Saving data files for date: '+date)

In [ ]:
# Function to call the REST API
def fetch_data_from_website(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    elif response.status_code == 404:
        print('No data found for Url: ' + url)
        return None
    else:
        print('Failed to fetch data from the url: ' + url)
        return None

In [ ]:
# Read Data from CSA Distributed compliance Ledger (Vendors)
url = 'https://on.dcl.csa-iot.org/dcl/vendorinfo/vendors?pagination.limit=10000'
data = fetch_data_from_website(url)
json_obj = json.loads(data)

# Parse the fetched data and convert it into a list of rows
df = pd.DataFrame(json_obj['vendorInfo'])

# Save in the CSV file
df.to_csv('vendors_'+date+'.csv', index=False)
print('Data saved to vendors_'+date+'.csv')

In [ ]:
# Read Data from CSA Distributed compliance Ledger (Models)
url = 'https://on.dcl.csa-iot.org/dcl/model/models?pagination.limit=10000'
response = fetch_data_from_website(url)
json_obj = json.loads(response)

# Parse the fetched data and convert it into a list of rows
df = pd.DataFrame(json_obj['model'])

# Save in the CSV file
df.to_csv('products_'+date+'.csv', index=False)
print('Data saved to products_'+date+'.csv')

In [ ]:
# Read Data from CSA Distributed compliance Ledger (Model Software Information)
#IDS: /VendorID/ProductID/Softwareversion
url = 'https://on.dcl.csa-iot.org/dcl/model/versions/'
productData = pd.read_csv('products_'+date+'.csv')
df= pd.DataFrame()
for index, product in productData.iterrows():
    print(product['pid'])
    response = fetch_data_from_website(url+str(product['vid'])+'/'+str(product['pid']))
    if(response != None):
        json_obj = json.loads(response)
        for version in json_obj['modelVersions']['softwareVersions']:
            response = fetch_data_from_website(url+str(product['vid'])+'/'+str(product['pid'])+'/'+str(version))
            if(response != None):
                json_obj = json.loads(response)
                # Parse the fetched data and convert it into a rows and add it to a dataframe
                df=pd.concat([df, pd.DataFrame([json_obj['modelVersion']])])
            else:
                print('Missing data for: '+str(product['vid'])+'/'+str(product['pid'])+'/'+str(version))

df.to_csv('modelVersions_'+date+'.csv', index=False)
print('Data saved to modelVersions_'+date+'.csv')

In [ ]:
# Read Data from CSA Distributed compliance Ledger (Compliance)
url = 'https://on.dcl.csa-iot.org/dcl/compliance/compliance-info'
response = fetch_data_from_website(url)
json_obj = json.loads(response)

# Parse the fetched data and convert it into a list of rows
df = pd.DataFrame(json_obj['complianceInfo'])

while(json_obj['pagination']['next_key'] != None):
    response = fetch_data_from_website(url+'?pagination.key='+urllib.parse.quote_plus(json_obj['pagination']['next_key']))
    json_obj = json.loads(response)
    
    # Parse the fetched data and convert it into a list of rows
    df2 = pd.DataFrame(json_obj['complianceInfo'])
    df=pd.concat([df, df2])

# Save in the CSV file
df.to_csv('compliance_'+date+'.csv', index=False)
print('Data saved to compliance_'+date+'.csv')